# 제 2회 기출문제

## 문제 1. 
주어진 데이터에서 'views' 컬럼을 기준으로 내림차순 정렬하고,<br>
'views' 컬럼 상위 10개 데이터를 'views' 컬럼 상위 10번째 값으로 대체한 후 <br>
'age' 컬럼이 80 이상인 데이터의 'views' 컬럼 평균값을 구하시오.(반올림하여 소수 둘째 자리까지 계산)



In [9]:
import pandas as pd

df = pd.read_csv('./ch2_data/members.csv')
df

,id,age,city,f1,f2,f3,f4,f5,subscribed,views
0,id01,2.0,서울,NaN,0,gold,ENFJ,91.297791,2021-07-16,6820.0
1,id02,9.0,서울,70.0,1,NaN,ENFJ,60.339826,2021-05-12,2534.0
2,id03,27.0,서울,61.0,1,gold,ISTJ,17.252986,2021-03-16,7312.0
3,id04,75.0,서울,NaN,2,NaN,INFP,52.667078,2021-07-21,493.0
4,id05,24.0,서울,85.0,2,NaN,ISFJ,29.269869,2021-03-07,1338.0
...,...,...,...,...,...,...,...,...,...,...
95,id95,77.0,경기,43.0,1,gold,INTJ,91.297791,2021-05-21,8697.0
96,id96,92.0,경기,53.0,1,silver,ENTJ,52.667078,2021-05-06,4336.0
97,id97,100.0,경기,NaN,0,gold,INFP,67.886373,2021-03-18,6687.0
98,id98,39.0,경기,58.0,2,NaN,INFP,98.429899,2021-10-02,865.0


In [12]:
# 1. views 컬럼 내림차순 정렬

df.sort_values('views', ascending=False, inplace=True)

# 2. 상위 10개 중에서 10번째 값 최솟값 구하기
min_value = df['views'][:10].min()

# 3. 상위 10개 중에서 10번째 값(최솟값) 대체
df.iloc[:10,-1] = min_value

# 4. age가 80이상의 views 컬럼 평균
conf = df['age']>=80
result = df[conf]['views'].mean()
print(round(result, 2))

5674.04


## 문제 2. 
주어진 데이터셋에서 앞에서부터 순서대로 80% 데이터만 활용해 <br>
'f1' 컬럼 결측치를 중앙 값으로 채우기 전후의 표준편차를 구하고 두 표본 표준편차의 차이를 계산하시오<br>
(두 표본 표준편차는 절대값으로 계산하고, 반올림하여 소수 둘째 자리까지 계산)


In [13]:
import pandas as pd
df = pd.read_csv('./ch2_data/members.csv')

In [16]:
line = int(len(df)*0.8)

df = df.iloc[:line]

In [18]:
std1 = df['f1'].std()
med = df['f1'].median()
df['f1'] = df['f1'].fillna(med)
std2 = df['f1'].std()

result = abs(std1-std2)
print(round(result,2))

3.56


## 문제 3.
주어진 데이터셋 'age' 컬럼의 이상치를 모두 더하시오.<br>
이상치는 평균으로부터 '표준편차*1.5'를 벗어나는 영역을 이상치라고 판단


In [19]:
import pandas as pd
df = pd.read_csv('./ch2_data/members.csv')

In [24]:
std1 = df['age'].std()
mean = df['age'].mean()

lower = mean - (std1 * 1.5)
upper = mean + (std1 * 1.5)
cond1 = df['age']<lower
cond2 = df['age']>upper

print(df[cond1|cond2]['age'].sum())

473.5


## 문제 4.
제품 배송 시간에 맞춰 배송되지 않을 확률값을 예측하시오.
- 예측할 컬럼 : Reached.on.Time_Y.N(0: 정시 도착, 1: 정시 도착 x)
- 평가 기준 : ROC_AUC로 평가


In [26]:
import pandas as pd

X_test = pd.read_csv('./ch2_data/X_test.csv')
X_train = pd.read_csv('./ch2_data/X_train.csv')
y_train = pd.read_csv('./ch2_data/y_train.csv')


In [29]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(8799, 11)
(2200, 11)
(8799, 2)


In [32]:
X_train.isnull().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64

In [33]:
X_test.isnull().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64

In [34]:
y_train['Reached.on.Time_Y.N'].value_counts()

Reached.on.Time_Y.N
1    5236
0    3563
Name: count, dtype: int64

In [38]:
from sklearn.preprocessing import LabelEncoder

cols = X_train.select_dtypes(include='object').columns
 

for col in cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])


In [39]:
X_train = X_train.drop('ID',axis=1)

In [40]:
X_test_id = X_test.pop('ID')

In [47]:
from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(
    X_train,
    y_train['Reached.on.Time_Y.N'],
    test_size=0.2,
    random_state=0
)


X_tr.shape,X_val.shape,y_tr.shape,y_val.shape

((7039, 10), (1760, 10), (7039,), (1760,))

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [49]:
# 로지스틱회귀
lr = LogisticRegression(random_state=0)
lr.fit(X_tr,y_tr)
pred = lr.predict_proba(X_val)
print(roc_auc_score(y_val, pred[:,1]))

0.7294862508383635


/Users/gihyun/Desktop/dev/GitHub/-BigDataAnalyst/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [50]:
dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_tr,y_tr)
pred = dt.predict_proba(X_val)
print(roc_auc_score(y_val,pred[:,1]))

0.6540107310529846


In [51]:
rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr,y_tr)
pred = rf.predict_proba(X_val)
print(roc_auc_score(y_val,pred[:,1]))

0.7657780013413816


In [53]:
pred = rf.predict_proba(X_test)
pred

array([[0.61, 0.39],
       [0.61, 0.39],
       [0.67, 0.33],
       ...,
       [0.58, 0.42],
       [0.66, 0.34],
       [0.4 , 0.6 ]], shape=(2200, 2))

In [54]:
submit = pd.DataFrame({
    'ID':X_test_id,
    'Reached.on.Time_Y.N':pred[:,1]
})

In [55]:
submit.head(5)

,ID,Reached.on.Time_Y.N
0,8285,0.39
1,10192,0.39
2,8675,0.33
3,5753,0.42
4,448,0.97


### 심화학습


In [70]:
import pandas as pd

X_test = pd.read_csv('./ch2_data/X_test.csv')
X_train = pd.read_csv('./ch2_data/X_train.csv')
y_train = pd.read_csv('./ch2_data/y_train.csv')


In [71]:
from sklearn.preprocessing import LabelEncoder
cols = X_train.select_dtypes(include='object').columns

for col in cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])


In [72]:
X_train = X_train.drop('ID',axis=1)
X_test_id = X_test.pop('ID')

In [76]:
from sklearn.preprocessing import MinMaxScaler

cols=['Customer_care_calls','Customer_rating','Cost_of_the_Product',
      'Prior_purchases','Discount_offered','Weight_in_gms'
      ]

scaler = MinMaxScaler()
X_train[cols] = scaler.fit_transform(X_train[cols])
X_test[cols] = scaler.fit_transform(X_test[cols])


In [77]:
from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(
    X_train,
    y_train['Reached.on.Time_Y.N'],
    test_size=0.2,
    random_state=0
)


X_tr.shape,X_val.shape,y_tr.shape,y_val.shape

((7039, 10), (1760, 10), (7039,), (1760,))

In [78]:
rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr,y_tr)
pred = rf.predict_proba(X_val)
print(roc_auc_score(y_val,pred[:,1]))

0.7657256874580818


# 제 3회 기출문제


## 1. 유형

### 문제1.
결측치가 있는 데이터의 행을 제거하고 앞에서부터 '70%' 데이터만 활용해<br>
'f1' 컬럼 1사분위 값을 구하시오.(소수점 제거 후 정수 출력)

In [90]:
import pandas as pd
df = pd.read_csv('./ch3_data/members.csv')

df = df.dropna()
lines = int(len(df)*0.7)
df = df.iloc[:lines]

print(int(df['f1'].quantile(.25)))

57


### 문제2.
2000년 데이터 중 2000년 평균보다 큰 값의 데이터 수를 구하시오
(index 연도)


In [91]:
import pandas as pd
df = pd.read_csv('./ch3_data/year.csv',index_col='Unnamed: 0')

In [ ]:
# 풀이 1. 행을 기준으로 평균 계산

m = df.loc[2000].mean()

print(sum(df.loc[2000,:]>m))

In [105]:
# 풀이 2. 행과 열을 변경해 평균 계산

df = df.T

m = df[2000].mean()
print(len(df[df[2000]>m]))

100


### 문제 3. 
결측치가 가장 많은 컬럼명을 출력하시오.

In [106]:
import pandas as pd
df = pd.read_csv("./ch3_data/members.csv")

In [109]:
df_cntNull = df.isnull().sum()

df_cntNull = df_cntNull.sort_values(ascending=False)
print(df_cntNull.index[0])

f1


### 문제 4. 
여행 보험 패키지 상품을 구매할 확률값을 예측하시오.<br>
예측할 컬럼 : Travellnsurance(패키지 구매 여부 0:구매 x, 1: 구매함) <br>
모델 성능은 roc-auc 평가지표

In [110]:
import pandas as pd
train = pd.read_csv('./ch3_data/train.csv')
test = pd.read_csv('./ch3_data/test.csv')

In [111]:
train.shape,test.shape

((1490, 10), (497, 9))

In [112]:
# train 결측치 x
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           1490 non-null   int64 
 1   Age                  1490 non-null   int64 
 2   Employment Type      1490 non-null   object
 3   GraduateOrNot        1490 non-null   object
 4   AnnualIncome         1490 non-null   int64 
 5   FamilyMembers        1490 non-null   int64 
 6   ChronicDiseases      1490 non-null   int64 
 7   FrequentFlyer        1490 non-null   object
 8   EverTravelledAbroad  1490 non-null   object
 9   TravelInsurance      1490 non-null   int64 
dtypes: int64(6), object(4)
memory usage: 116.5+ KB


In [114]:
# test 결측치 x
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           497 non-null    int64 
 1   Age                  497 non-null    int64 
 2   Employment Type      497 non-null    object
 3   GraduateOrNot        497 non-null    object
 4   AnnualIncome         497 non-null    int64 
 5   FamilyMembers        497 non-null    int64 
 6   ChronicDiseases      497 non-null    int64 
 7   FrequentFlyer        497 non-null    object
 8   EverTravelledAbroad  497 non-null    object
dtypes: int64(5), object(4)
memory usage: 35.1+ KB


In [115]:
train['TravelInsurance'].value_counts()

TravelInsurance
0    965
1    525
Name: count, dtype: int64

In [116]:
train.describe(include='object')

,Employment Type,GraduateOrNot,FrequentFlyer,EverTravelledAbroad
count,1490,1490,1490,1490
unique,2,2,2,2
top,Private Sector/Self Employed,Yes,No,No
freq,1056,1270,1175,1209


In [118]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           1490 non-null   int64 
 1   Age                  1490 non-null   int64 
 2   Employment Type      1490 non-null   object
 3   GraduateOrNot        1490 non-null   object
 4   AnnualIncome         1490 non-null   int64 
 5   FamilyMembers        1490 non-null   int64 
 6   ChronicDiseases      1490 non-null   int64 
 7   FrequentFlyer        1490 non-null   object
 8   EverTravelledAbroad  1490 non-null   object
 9   TravelInsurance      1490 non-null   int64 
dtypes: int64(6), object(4)
memory usage: 116.5+ KB


In [119]:
target = train.pop('TravelInsurance')

In [120]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [121]:
from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(
    train,
    target,
    test_size=0.2,
    random_state=0
)
X_tr.shape,X_val.shape,y_tr.shape,y_val.shape

((1192, 13), (298, 13), (1192,), (298,))

In [123]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [124]:
lr = LogisticRegression(random_state=0)

lr.fit(X_tr,y_tr)
pred = lr.predict_proba(X_val)
print(roc_auc_score(y_val,pred[:,1]))

0.7745126705653022


/Users/gihyun/Desktop/dev/GitHub/-BigDataAnalyst/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [126]:
dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_tr,y_tr)
pred = dt.predict_proba(X_val)
print(roc_auc_score(y_val, pred[:,1]))

0.7090155945419103


In [127]:
rf = RandomForestClassifier(random_state=0)

rf.fit(X_tr,y_tr)
pred = rf.predict_proba(X_val)
print(roc_auc_score(y_val,pred[:,1]))

0.850682261208577


In [128]:
pred = rf.predict_proba(test)
pred

array([[0.76, 0.24],
       [0.93, 0.07],
       [0.89, 0.11],
       [0.02, 0.98],
       [0.72, 0.28],
       [0.  , 1.  ],
       [0.53, 0.47],
       [0.93, 0.07],
       [0.22, 0.78],
       [0.95, 0.05],
       [0.93, 0.07],
       [0.92, 0.08],
       [0.7 , 0.3 ],
       [0.93, 0.07],
       [0.09, 0.91],
       [0.86, 0.14],
       [0.58, 0.42],
       [0.85, 0.15],
       [0.6 , 0.4 ],
       [0.38, 0.62],
       [0.88, 0.12],
       [0.88, 0.12],
       [0.  , 1.  ],
       [0.04, 0.96],
       [1.  , 0.  ],
       [0.01, 0.99],
       [0.01, 0.99],
       [0.86, 0.14],
       [0.02, 0.98],
       [0.01, 0.99],
       [0.91, 0.09],
       [0.73, 0.27],
       [0.9 , 0.1 ],
       [0.68, 0.32],
       [0.88, 0.12],
       [0.13, 0.87],
       [0.87, 0.13],
       [0.82, 0.18],
       [0.04, 0.96],
       [0.79, 0.21],
       [0.91, 0.09],
       [1.  , 0.  ],
       [0.69, 0.31],
       [0.74, 0.26],
       [0.08, 0.92],
       [0.86, 0.14],
       [0.02, 0.98],
       [0.74,

In [131]:
submit = pd.DataFrame({
    'index':test.index,
    'y_pred':pred[:,1]})

In [133]:
# submit.to_csv('./ch3_data/result.csv',index=False)

#### 심화 
기존 점수 : 0.85
RandomForestClassifier

In [ ]:
import pandas as pd

train = pd.read_csv('./ch3_data/train.csv')
test = pd.read_csv('./ch3_data/test.csv')

In [135]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           1490 non-null   int64 
 1   Age                  1490 non-null   int64 
 2   Employment Type      1490 non-null   object
 3   GraduateOrNot        1490 non-null   object
 4   AnnualIncome         1490 non-null   int64 
 5   FamilyMembers        1490 non-null   int64 
 6   ChronicDiseases      1490 non-null   int64 
 7   FrequentFlyer        1490 non-null   object
 8   EverTravelledAbroad  1490 non-null   object
 9   TravelInsurance      1490 non-null   int64 
dtypes: int64(6), object(4)
memory usage: 116.5+ KB


In [136]:
target = train.pop('TravelInsurance')

In [138]:
from sklearn.preprocessing import LabelEncoder

cols = train.select_dtypes(include='O').columns
for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])


In [140]:
from sklearn.model_selection import train_test_split

X_tr,X_val,y_tr,y_val = train_test_split(
    train,target,test_size=0.2, random_state=0
)
X_tr.shape,X_val.shape,y_tr.shape,y_val.shape

((1192, 9), (298, 9), (1192,), (298,))

In [142]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

rf = RandomForestClassifier(random_state=0)

rf.fit(X_tr,y_tr)
pred = rf.predict_proba(X_val)
print(roc_auc_score(y_val,pred[:,1]))

0.8630847953216375


#### 심화 
기존 점수 : 0.85 RandomForestClassifier<br>
LabelEncoder -> 0.8630847953216375<br>
스케일링(로버스트) : 0.8631091617933725<br>


In [145]:
import pandas as pd

train = pd.read_csv('./ch3_data/train.csv')
test = pd.read_csv('./ch3_data/test.csv')
target = train.pop('TravelInsurance')


from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
cols =['Age','AnnualIncome','FamilyMembers','ChronicDiseases']

train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])


from sklearn.preprocessing import LabelEncoder

cols = train.select_dtypes(include='O').columns
for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])


from sklearn.model_selection import train_test_split

X_tr,X_val,y_tr,y_val = train_test_split(
    train,target,test_size=0.2, random_state=0
)
X_tr.shape,X_val.shape,y_tr.shape,y_val.shape


from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

rf = RandomForestClassifier(random_state=0)

rf.fit(X_tr,y_tr)
pred = rf.predict_proba(X_val)
print(roc_auc_score(y_val,pred[:,1]))

0.8631091617933725


In [148]:
pred = rf.predict_proba(test)
submit = pd.DataFrame({
    'index':test.index,
    'pred':pred[:,1]})

# 제 4회 기출문제


## 1유형


### 문제
'age' 컬럼의 3사분위수와 1분위수의 차이를 절대값으로 구하고, 정수로 출력


In [150]:
import pandas as pd
df = pd.read_csv('./ch4_data/data4-1.csv')

In [152]:
q1 = df['age'].quantile(0.25)
q3 = df['age'].quantile(0.75)


In [154]:
print(int(abs(q1-q3)))

50


### 문제

모든 reactions 중 loves 와 wows를 합한 반응 비율이 40%보다 크고 50%보다 작은 데이터를 찾고<br>
이 중 type컬럼이 'video'인 데이터의 개수를 구하시오.

In [155]:
import pandas as pd
df = pd.read_csv('./ch4_data/data4-2.csv')
df

,id,type,reactions,comments,shares,likes,loves,wows,hahas,sads,angrys
0,1,video,529,512,262,432,92,3,1,1,0
1,2,photo,150,0,0,150,0,0,0,0,0
2,3,video,227,236,57,204,21,1,1,0,0
3,4,photo,111,0,0,111,0,0,0,0,0
4,5,photo,213,0,0,204,9,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
7045,7046,photo,89,0,0,89,0,0,0,0,0
7046,7047,photo,16,0,0,14,1,0,1,0,0
7047,7048,photo,2,0,0,1,1,0,0,0,0
7048,7049,photo,351,12,22,349,2,0,0,0,0


In [159]:
cond1 = (df['loves']+df['wows'])/df['reactions']>0.4
cond2 = (df['loves']+df['wows'])/df['reactions']<0.5
cond3 = df['type']=='video'

In [160]:
print(len(df[cond1&cond2&cond3]))

90


### 문제
'data_added' 컬럼의 날짜가 2018년 1월이면서 'country'에서 United Kingdom 단독 제작인 데이터의 개수

In [177]:
import pandas as pd
df = pd.read_csv("./ch4_data/data4-3.csv")
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries"
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act..."
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV"
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ..."
...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers"
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies"
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies"
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies"


In [188]:
df['date_added'] = pd.to_datetime(df['date_added'])

In [191]:
df['year'] = df['date_added'].dt.year
df['month'] = df['date_added'].dt.month

In [194]:
cond1 = df['country'] == 'United Kingdom'
cond2 = df['date_added'] >='2018-1-1'
cond3 = df['date_added'] <='2018-1-31'

print(len(df[cond1&cond2&cond3]))

6


## 2유형

### 문제
고객의 개인 정보와 행동 패턴을 기반으로 고객의 세분화 결과를 예측하시오.
- 예측할 컬럼 : Segmentation(1,2,3,4)
- 제출파일
    - ID : 고객 식별자
    - Segmentation(1,2,3,4)
- 모델의 성능 macro F1 score

In [195]:
import pandas as pd
train = pd.read_csv('./ch4_data/train.csv')
test = pd.read_csv('./ch4_data/test.csv')

In [ ]:
train.info()
# 결측치 x
# 범주형 6개
# 수치형 5개

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               6665 non-null   int64  
 1   Gender           6665 non-null   object 
 2   Ever_Married     6665 non-null   object 
 3   Age              6665 non-null   int64  
 4   Graduated        6665 non-null   object 
 5   Profession       6665 non-null   object 
 6   Work_Experience  6665 non-null   float64
 7   Spending_Score   6665 non-null   object 
 8   Family_Size      6665 non-null   float64
 9   Var_1            6665 non-null   object 
 10  Segmentation     6665 non-null   int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 572.9+ KB


In [ ]:
test.info()

# 결측치 x
# 범주형 ; 6
# 수치형 : 4

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2154 entries, 0 to 2153
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               2154 non-null   int64  
 1   Gender           2154 non-null   object 
 2   Ever_Married     2154 non-null   object 
 3   Age              2154 non-null   int64  
 4   Graduated        2154 non-null   object 
 5   Profession       2154 non-null   object 
 6   Work_Experience  2154 non-null   float64
 7   Spending_Score   2154 non-null   object 
 8   Family_Size      2154 non-null   float64
 9   Var_1            2154 non-null   object 
dtypes: float64(2), int64(2), object(6)
memory usage: 168.4+ KB


In [198]:
train.describe()

,ID,Age,Work_Experience,Family_Size,Segmentation
count,6665.00000,6665.000000,6665.000000,6665.000000,6665.000000
mean,463519.84096,43.536084,2.629107,2.841110,2.542836
std,2566.43174,16.524054,3.405365,1.524743,1.122723
min,458982.00000,18.000000,0.000000,1.000000,1.000000
25%,461349.00000,31.000000,0.000000,2.000000,2.000000
50%,463575.00000,41.000000,1.000000,2.000000,3.000000
75%,465741.00000,53.000000,4.000000,4.000000,4.000000
max,467974.00000,89.000000,14.000000,9.000000,4.000000


In [ ]:
train.describe(include='O')

,Gender,Ever_Married,Graduated,Profession,Spending_Score,Var_1
count,6665,6665,6665,6665,6665,6665
unique,2,2,2,9,3,7
top,Male,Yes,Yes,Artist,Low,Cat_6
freq,3677,3944,4249,2192,3999,4476


In [200]:
target = train.pop("Segmentation")

In [201]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(
    train,target, test_size=0.2, random_state=0
)

X_tr.shape, X_val.shape,y_tr.shape,y_val.shape

((5332, 29), (1333, 29), (5332,), (1333,))

In [202]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

In [ ]:
# 0.33130502629966674
lr = LogisticRegression(random_state=0)
lr.fit(X_tr,y_tr)
pred = lr.predict(X_val)
print(f1_score(y_val,pred,average='macro'))

0.33130502629966674


/Users/gihyun/Desktop/dev/GitHub/-BigDataAnalyst/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# 0.4368478801889553
dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_tr,y_tr)
pred = dt.predict(X_val)
print(f1_score(y_val,pred,average='macro'))

0.4368478801889553


In [208]:
# 0.5097607361203126
rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr,y_tr)
pred = rf.predict(X_val)
print(f1_score(y_val,pred,average='macro'))

0.5097607361203126


In [212]:
pred = rf.predict(test)
submit = pd.DataFrame({
    'ID':test['ID'],
    'Segmentation':pred
})

### 데이터 전처리 및 심화




In [215]:
import pandas as pd

train = pd.read_csv("./ch4_data/train.csv")
test = pd.read_csv("./ch4_data/test.csv")
target = train.pop('Segmentation')

train.drop('ID',axis=1, inplace=True)
test_ID = test.pop('ID')

train = pd.get_dummies(train)
test = pd.get_dummies(test)

from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(
    train,
    target,
    test_size=0.2,
    random_state=0
)

X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)

rf.fit(X_tr,y_tr)
pred = rf.predict(X_val)
print(f1_score(y_val,pred,average='macro'))

0.4860743688384891


In [216]:
import pandas as pd

train = pd.read_csv("./ch4_data/train.csv")
test = pd.read_csv("./ch4_data/test.csv")
target = train.pop('Segmentation')

train.drop('ID',axis=1, inplace=True)
test_ID = test.pop('ID')

from sklearn.preprocessing import LabelEncoder
cols = train.select_dtypes(include='O').columns

for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(
    train,
    target,
    test_size=0.2,
    random_state=0
)

X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)

rf.fit(X_tr,y_tr)
pred = rf.predict(X_val)
print(f1_score(y_val,pred,average='macro'))

0.4940937547227707


In [218]:
import pandas as pd

train = pd.read_csv("./ch4_data/train.csv")
test = pd.read_csv("./ch4_data/test.csv")
target = train.pop('Segmentation')

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
cols =['ID','Age','Work_Experience','Family_Size']

train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])


train = pd.get_dummies(train)
test = pd.get_dummies(test)

from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(
    train,
    target,
    test_size=0.2,
    random_state=0
)

X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)

rf.fit(X_tr,y_tr)
pred = rf.predict(X_val)
print(f1_score(y_val,pred,average='macro'))

0.509334828690811


In [219]:
pred = rf.predict(test)
submit = pd.DataFrame({
    'ID':test['ID'],
    'Segmentation':pred
})

# 제 5회 기출문제


## 1유형


### 문제
종량제 봉투 종류가 '규격봉투'고 종량제 봉투 용도가 '음식물쓰레기'인 2L 가격 평균을 구하시오.<br>
단, 가격이 0이면 평균에서 제외하고, 반올림 후 정수를 출력하시오.

In [220]:
import pandas as pd

df = pd.read_csv('./ch5_data/data5-1.csv')

In [226]:
cond1 = df['종량제봉투종류']=='규격봉투'
cond2 = df['종량제봉투용도']=='음식물쓰레기'
cond3 = df['2ℓ가격']>0

df = df[cond1&cond2&cond3]

print(round(df['2ℓ가격'].mean()))

118


### 문제
bmi 수치가 정상인 사람 수와 위험체중인 사람 수의 차이를 절대값으로 구하시오(정수)


In [229]:
import pandas as pd
df = pd.read_csv('./ch5_data/data5-2.csv')
df['bmi'] = df['Weight'] / ((df['Height'] * 0.01)**2)

In [231]:
# 정상 체중
cond1 = (df['bmi'] >=18.5)&(df['bmi']<23)
cond2 = (df['bmi']>=23) & (df['bmi']<25)

print(abs(len(df[cond1])-len(df[cond2])))

144


### 문제
순 전입학생이 가장 많은 학교의 전체 학생수를 구하시오.<br>
(순 전입학생 = 전입학생 - 전출학생)

In [239]:
import pandas as pd
df = pd.read_csv('./ch5_data/data5-3.csv')
df['순 전입학생'] = df['전입학생수(계)']-df['전출학생수(계)']

In [241]:
df = df.sort_values('순 전입학생', ascending=False)

In [243]:
int(df.iloc[0,-2])

230

## 2유형

### 문제.
차량의 특징 기반 데이터를 활용해 test 데이터의 중고차 가격을 예측하시오.<br>

- 예측할 컬럼 : price
- 모델 성능 rmse(Root Mean Square Error) 평가지표에 따라 채점


In [253]:
import pandas as pd
train = pd.read_csv('./ch5_data/train.csv')
test = pd.read_csv('./ch5_data/test.csv')

In [254]:
train.shape, test.shape

((3759, 9), (1617, 8))

In [255]:
# 결측치 x
# 범주형 3개
# 수치형 6개
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3759 entries, 0 to 3758
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         3759 non-null   object 
 1   year          3759 non-null   int64  
 2   price         3759 non-null   int64  
 3   transmission  3759 non-null   object 
 4   mileage       3759 non-null   int64  
 5   fuelType      3759 non-null   object 
 6   tax           3759 non-null   int64  
 7   mpg           3759 non-null   float64
 8   engineSize    3759 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 264.4+ KB


In [256]:
# 결측치 x
# 범주형 3개
# 수치형 5개
test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1617 entries, 0 to 1616
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         1617 non-null   object 
 1   year          1617 non-null   int64  
 2   transmission  1617 non-null   object 
 3   mileage       1617 non-null   int64  
 4   fuelType      1617 non-null   object 
 5   tax           1617 non-null   int64  
 6   mpg           1617 non-null   float64
 7   engineSize    1617 non-null   float64
dtypes: float64(2), int64(3), object(3)
memory usage: 101.2+ KB


In [257]:
train.describe(include='O')

,model,transmission,fuelType
count,3759,3759,3759
unique,19,3,3
top,Fiesta,Manual,Petrol
freq,1372,3234,2541


In [258]:
target = train.pop('price')
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [259]:
from sklearn.model_selection import train_test_split

X_tr,X_val,y_tr,y_val = train_test_split(
    train,target, test_size=0.2, random_state=0
)

X_tr.shape, X_val.shape, y_tr.shape,y_val.shape

((3007, 30), (752, 30), (3007,), (752,))

In [264]:
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor


In [265]:
lr = LogisticRegression()
lr.fit(X_tr,y_tr)
pred = lr.predict(X_val)
print(root_mean_squared_error(y_val,pred))

4018.173528026109


/Users/gihyun/Desktop/dev/GitHub/-BigDataAnalyst/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [266]:
rf = RandomForestRegressor(random_state=0)
rf.fit(X_tr,y_tr)
pred = rf.predict(X_val)
print(root_mean_squared_error(y_val,pred))

1341.943732584459


#### 성능 개선

In [270]:
import pandas as pd
train = pd.read_csv("./ch5_data/train.csv")
test = pd.read_csv('./ch5_data/test.csv')

target = train.pop('price')

from sklearn.preprocessing import LabelEncoder
cols = train.select_dtypes(include='O').columns
for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(
    train, target, test_size =0.2, random_state=0
)

In [271]:
rf = RandomForestRegressor(random_state=0)
rf.fit(X_tr,y_tr)
pred = rf.predict(X_val)
print(root_mean_squared_error(y_val,pred))

1302.5271377093507


In [272]:
pred = rf.predict(test)
result = pd.DataFrame({
    'pred':pred
})

In [273]:
result

,pred
0,15622.74
1,15892.63
2,14940.32
3,18392.95
4,6164.23
...,...
1612,10887.99
1613,17342.15
1614,9529.61
1615,14371.82


# 제 6회 기출문제


## 제 1유형

### 문제
출동 시간과 도착시간 차이가 평균적으로 가장 오래 걸린 소방서의 시간을 분으로 변환해 출력하시오.

In [ ]:
import pandas as pd
df = pd.read_csv('./ch6_data/data6-1-1.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   소방서     100 non-null    object
 1   출동시간    100 non-null    object
 2   도착시간    100 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


In [276]:
df['출동시간'] = pd.to_datetime(df['출동시간'])
df['도착시간'] = pd.to_datetime(df['도착시간'])

In [278]:
df['diff'] = df['도착시간'] - df['출동시간']
df['diff'] = df['diff'].dt.total_seconds()/60

In [280]:
avg_diff = df.groupby('소방서')['diff'].mean()
idx = avg_diff.idxmax()

In [281]:
result = avg_diff[idx]
print(round(result))

81


### 문제
- 학교에서 교사 한 명당 맡은 학생수가 가장 많은 학교를 찾고, 그 학교의 전체 교사 수를 구하시오.

In [285]:
import pandas as pd
df = pd.read_csv("./ch6_data/data6-1-2.csv")
df['총학생수'] = df.iloc[:,2:].sum(axis=1)

In [286]:
df

,학교명,교사수,1학년,2학년,3학년,4학년,5학년,6학년,총학생수
0,학교1,20,118,72,154,118,77,59,598
1,학교2,31,187,193,135,137,63,90,805
2,학교3,23,136,193,135,112,91,50,717
3,학교4,32,72,146,67,51,130,164,630
4,학교5,22,61,73,86,82,55,124,481
...,...,...,...,...,...,...,...,...,...
95,학교96,38,75,166,113,191,172,57,774
96,학교97,30,125,169,145,122,187,194,942
97,학교98,37,136,100,93,60,145,63,597
98,학교99,37,74,175,145,157,95,76,722


In [288]:
df['학생/교사'] = df['총학생수']/df['교사수']

idx = df['학생/교사'].idxmax()

In [289]:
print(df.loc[idx,'교사수'])

19


### 문제
연도별로 총 범죄 건수(범죄 유형별 발생 건수의 합)의 월 평균 값을 구한 후 <br>
그 값이 가장 큰 연도를 찾아 해당 연도의 총 범죄 건수의 월평균 값을 출력

In [290]:
import pandas as pd
df = pd.read_csv("./ch6_data/data6-1-3.csv")

In [300]:
# df.iloc[:,1:-1]

df['총범죄수'] = df.iloc[:,1:-1].sum(axis=1)

In [301]:
df['연도'] = df['날짜'].str[:4]

In [303]:
result = df['총범죄수'].groupby(df['연도']).sum()
print(round(result.max()/12))

533


## 2유형

### 문제
건물 데이터를 기반으로 건물의 난방 부하(에너지 효율성)을 예측하시오.<br>
- 예측할 컬럼 : Heat_Load(Very Low,Low, Medium, High, Very High)
- 모델의 성능은 Macro F1-score평가지표에 따라 채점

In [304]:
import pandas as pd
train = pd.read_csv('./ch6_data/energy_train.csv')
test = pd.read_csv('./ch6_data/energy_test.csv')

train.shape, test.shape

((537, 10), (231, 9))

In [ ]:
train.info()
# 결측치 x
# 수치형 컬럼 : 6개
# 범주형 컬럼 : 4개

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537 entries, 0 to 536
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Compac       537 non-null    float64
 1   Surf_Area    537 non-null    float64
 2   Wall_Area    537 non-null    float64
 3   Roof         537 non-null    object 
 4   Height       537 non-null    object 
 5   Orient       537 non-null    object 
 6   Glaze_Area   537 non-null    float64
 7   Glaze_Distr  537 non-null    int64  
 8   Cool_Load    537 non-null    float64
 9   Heat_Load    537 non-null    object 
dtypes: float64(5), int64(1), object(4)
memory usage: 42.1+ KB


In [307]:
test.info()
# 결측치 x
# 수치형 컬럼 : 6개
# 범주형 컬럼 : 3개

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Compac       231 non-null    float64
 1   Surf_Area    231 non-null    float64
 2   Wall_Area    231 non-null    float64
 3   Roof         231 non-null    object 
 4   Height       231 non-null    object 
 5   Orient       231 non-null    object 
 6   Glaze_Area   231 non-null    float64
 7   Glaze_Distr  231 non-null    int64  
 8   Cool_Load    231 non-null    float64
dtypes: float64(5), int64(1), object(3)
memory usage: 16.4+ KB


In [311]:
train.describe()

,Compac,Surf_Area,Wall_Area,Glaze_Area,Glaze_Distr,Cool_Load
count,537.00000,537.000000,537.000000,537.000000,537.000000,537.000000
mean,0.76879,667.796089,318.271881,0.237244,2.808194,24.939628
std,0.10636,88.123635,42.447824,0.132183,1.541251,9.600170
min,0.62000,514.500000,245.000000,0.000000,0.000000,11.170000
25%,0.69000,588.000000,294.000000,0.100000,2.000000,15.770000
50%,0.76000,661.500000,318.500000,0.250000,3.000000,25.160000
75%,0.86000,735.000000,343.000000,0.400000,4.000000,33.540000
max,0.98000,808.500000,416.500000,0.400000,5.000000,48.030000


In [317]:
train.describe(include='O')

,Roof,Height,Orient,Heat_Load
count,537,537,537,537
unique,4,2,4,5
top,220.5,Tall,South,Very Low
freq,257,280,145,142


In [319]:
target = train.pop("Heat_Load")

In [320]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [321]:
from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(
    train,target,test_size=0.2, random_state=0
)

X_tr.shape, X_val.shape,y_tr.shape, y_val.shape

((429, 16), (108, 16), (429,), (108,))

In [323]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

In [ ]:
dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_tr,y_tr)
pred = dt.predict(X_val)
print(f1_score(y_val,pred, average='macro'))

0.9207614607614607


In [327]:
rt = RandomForestClassifier(random_state=0)
rt.fit(X_tr,y_tr)
pred = rt.predict(X_val)
print(f1_score(y_val, pred,average='macro'))


0.9277616846430405


In [329]:
pred = rt.predict(test)
submit = pd.DataFrame({
    'pred':pred
})

submit

,pred
0,Low
1,High
2,High
3,Low
4,Low
...,...
226,Very Low
227,Medium
228,Very Low
229,Low


## 3유형

### 문제
감기약 위약을 투여받은 환자의 부작용은 항암약 위약을 투여받은 환자의 부작용과 차이가 있는가?

1. 아픔 (비율 : 10%)
2. 조금 아픔 (비율 : 5%)
3. 속쓰림 (비율 : 15%)
4. 이상없음 (비율 : 70%)



In [1]:
import pandas as pd
df = pd.DataFrame({
    '항암약':[4,4,3,4,1,4,1,4,1,4,4,2,1,4,2,3,2,4,4,4]
})

print(df.head(3))

   항암약
0    4
1    4
2    3


- 귀무가설 : 감기약(위약)의 부작용과 항암약(위약)의 부작용은 동일하다.
- 대립가설 : 감기약(위약)의 부작용과 항암약(위약)의 부작용은 다르다.

1. 문제 항암약을 투여받은 환자 중 '이상없음'의 비율을 0과 1사이로 구하시오.
2. 감기약의 예상 부작용 비율과 항암약의 부작용 관찰값이 통게적으로 유의미하게 차이가 있는지 확인하려 한다.(카이제곱 검정을 사용해 검정 통계량을 구하시오.)
3. 위의 p-value를 구하시오

In [2]:
cnt = sum(df['항암약']==4)

ratio = cnt/len(df)
print(ratio)

0.55


In [5]:
# 1. 아픔 (비율 : 10%)
# 2. 조금 아픔 (비율 : 5%)
# 3. 속쓰림 (비율 : 15%)
# 4. 이상없음 (비율 : 70%)
from scipy.stats import chisquare
prob = [0.1,0.05, 0.15, 0.7]

print("데이터 수 : " ,len(df))

expected_counts = [0.1*20,0.05*20,0.15*20,0.7*20]
print(expected_counts)

expected_counts = [x*len(df) for x in prob]
print(expected_counts)

observed_counts = df['항암약'].value_counts().sort_index().to_list()
print(observed_counts)

print(chisquare(expected_counts,observed_counts))

데이터 수 :  20
[2.0, 1.0, 3.0, 14.0]
[2.0, 1.0, 3.0, 14.0]
[4, 3, 2, 11]
Power_divergenceResult(statistic=np.float64(3.6515151515151514), pvalue=np.float64(0.30163621963367865))


### 문제
- 독립변수 : solar(태양 에너지), wind(바람의 세기), O3(오존 농도)
- 종속변수 : temperature(온도)

In [7]:
import pandas as pd
df = pd.read_csv('./ch6_data/data6-3-2.csv')
df.head(3)

,solar,wind,o3,temperature
0,89.14,6.28,33.52,23.0
1,109.97,1.04,27.01,20.7
2,102.83,6.42,41.00,20.5


1. 주어진 데이터를 기반으로 다중 선형 회귀 모델을 구축하고, 독립변수 o3의 회귀계수를 구하시오
2. 문제 1에서 적합한 모델 'solar'와 o3이 고정된 상태에서 wind의 세기가 증가함에 따라 temperature가 감소한다.<br>
    이 회귀 모델에서 wind의 회귀계수에 대한 p-value 값을 구하시오.
3. 문제 2에서 적합한 모델에서 solar : 100, wind :5, o3:30일때 예측값을 구하시오.

In [10]:
from statsmodels.formula.api import ols

formula = 'temperature ~ solar + wind + o3'
model = ols(formula, df).fit()
summary = model.summary()
print(summary)

                            OLS Regression Results                            
Dep. Variable:            temperature   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     1.464
Date:                Sun, 23 Nov 2025   Prob (F-statistic):              0.229
Time:                        22:23:56   Log-Likelihood:                -195.45
No. Observations:                 100   AIC:                             398.9
Df Residuals:                      96   BIC:                             409.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     19.0507      1.994      9.555      0.0

In [12]:
print('1번 답 ', model.params['o3'])

1번 답  0.07493854378136566


In [13]:
print("2번 문제", model.pvalues['wind'])

2번 문제 0.779717720207169


In [15]:
new_data = pd.DataFrame({
    'solar' : [100], 'wind' :[5], 'o3':[30]
})

model.predict(new_data)

0    21.56163
dtype: float64

# 제 7회 기출문제

## 1유형


### 문제
결측치가 있는 행을 제거한 후 학생이 가장 많이 수강한 과목을 찾고, 해당 과목 점수를 표준화(스탠더드 스케일)한 다음 가장 큰 표준화 된 값을 구하시오.

In [42]:
import pandas as pd
df = pd.read_csv('./ch7_data/student_assessment.csv')
df

,id_assessment,student_id,study_period_days,score
0,103,0,22,87.0
1,93,1,70,92.0
2,15,2,22,2.0
3,107,3,22,51.0
4,72,4,22,73.0
...,...,...,...,...
2560,96,2695,35,3.0
2561,38,2696,22,77.0
2562,87,2697,22,88.0
2563,88,2698,70,67.0


In [43]:
df.dropna(inplace=True)

In [44]:
id = df['id_assessment'].value_counts().idxmax()


In [45]:
cond = df['id_assessment'] == id
df = df[cond]

In [48]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df['score'] = scaler.fit_transform(df[['score']])

/var/folders/6c/5zw1q8hx6r51frqls4xy7vdc0000gn/T/ipykernel_1511/427871623.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score'] = scaler.fit_transform(df[['score']])


In [49]:
print(round(df['score'].max(),3))

2.183


### 문제
DE1~DE77 컬럼 중 주가지수의 종가 'close'와 가장 상관관계가 높은 벼수를 찾아 해당 변수의 평균값을 구하시오.

In [1]:
import pandas as pd
df = pd.read_csv("./ch7_data/stock_market.csv")
df

,DE1,DE2,DE3,DE4,DE5,DE6,DE7,DE8,DE9,DE10,...,DE71,DE72,DE73,DE74,DE75,DE76,DE77,open,close,volume
0,1.764052,0.400157,0.978738,2.240893,1.867558,-0.977278,0.950088,-0.151357,-0.103219,0.410599,...,0.729091,0.128983,1.139401,-1.234826,0.402342,-0.684810,-0.870797,129.549102,126.502056,981466.0
1,-0.578850,-0.311553,0.056165,-1.165150,0.900826,0.465662,-1.536244,1.488252,1.895889,1.178780,...,1.117016,-1.315907,-0.461585,-0.068242,1.713343,-0.744755,-0.826439,126.502056,128.041811,473416.0
2,-0.098453,-0.663478,1.126636,-1.079932,-1.147469,-0.437820,-0.498032,1.929532,0.949421,0.087551,...,-1.540797,0.063262,0.156507,0.232181,-0.597316,-0.237922,-1.424061,128.041811,128.957651,46597.0
3,-0.493320,-0.542861,0.416050,-1.156182,0.781198,1.494485,-2.069985,0.426259,0.676908,-0.637437,...,1.658131,-0.118164,-0.680178,0.666383,-0.460720,-1.334258,-1.346718,128.957651,122.965246,455747.0
4,0.693773,-0.159573,-0.133702,1.077744,-1.126826,-0.730678,-0.384880,0.094352,-0.042171,-0.286887,...,-0.206904,0.880179,-1.698106,0.387280,-2.255564,-1.022507,0.038631,122.965246,114.922790,242198.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.020937,-0.942656,-0.437589,0.400197,0.280305,-0.239677,-0.523317,-0.070756,0.813117,-0.394868,...,-0.658888,0.129819,0.343603,0.240547,0.869118,-0.200800,-2.477692,50.254255,31.155158,955899.0
996,0.116310,-0.387522,-0.135163,0.427945,0.020041,-1.281488,-0.003546,0.517104,-0.798964,-0.590362,...,-0.297669,0.652641,0.807874,-2.239079,-1.378841,-0.020303,1.058002,31.155158,35.253943,646770.0
997,-0.167591,-0.304002,0.407705,1.625603,-0.480631,1.546496,-0.288261,-0.625352,1.186237,0.270277,...,0.200219,0.242471,2.453099,0.111067,-1.904594,0.643397,0.416017,35.253943,44.833041,887211.0
998,-1.780806,-0.133918,0.825544,-0.223826,0.426823,1.024924,0.847111,1.370670,1.187785,-0.938389,...,0.946004,-0.418479,-1.790980,-1.013184,-0.316554,-2.145770,-0.987003,44.833041,56.070234,174999.0


In [11]:
df_corr = df.corr()['close'].abs()

col = df_corr.loc['DE1':'DE77'].idxmax()
print(round(df[col].mean(),4))

-0.0004


### 문제
IQR을 이용해 이산화탄소(CO2) 이상치 수를 찾으시오.

In [13]:
import pandas as pd
df = pd.read_csv('./ch7_data/air_quality.csv')
df

,Date,CO,CO2,O2
0,2024-01-01 00:00:00,0.549671,383.037632,21.248286
1,2024-01-01 01:00:00,0.486174,392.362513,21.183324
2,2024-01-01 02:00:00,0.564769,385.065473,19.963480
3,2024-01-01 03:00:00,0.652303,402.760451,21.479584
4,2024-01-01 04:00:00,0.476585,429.929463,19.409917
...,...,...,...,...
9995,2025-02-20 11:00:00,0.630110,409.695684,20.322712
9996,2025-02-20 12:00:00,0.300166,434.434164,22.344604
9997,2025-02-20 13:00:00,0.429468,409.458849,20.398633
9998,2025-02-20 14:00:00,0.549577,442.838243,19.007643


In [16]:
q1 = df['CO2'].quantile(0.25)
q3 = df['CO2'].quantile(0.75)

In [19]:
IQR = q3-q1
upper = q3 + 1.5*IQR
lower = q1 - 1.5*IQR

cond1 = df['CO2']<lower
cond2 = df['CO2']>upper

print(len(df[cond1|cond2]))

304


## 2유형


### 문제
W-mart 매장의 판매 데이터를 기반으로 판매액을 예측하시오.
- 예측할 컬럼 : total
- RMSE 평가지표에 따라 채점

In [20]:
import pandas as pd
train = pd.read_csv('./ch7_data/mart_train.csv')
test = pd.read_csv('./ch7_data/mart_test.csv')
train.shape, test.shape

((700, 10), (300, 9))

In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   branch          700 non-null    object 
 1   city            700 non-null    object 
 2   customer_type   700 non-null    object 
 3   gender          700 non-null    object 
 4   product_line    700 non-null    object 
 5   total           700 non-null    float64
 6   payment_method  700 non-null    object 
 7   rating          700 non-null    float64
 8   time_of_day     700 non-null    object 
 9   day_name        700 non-null    object 
dtypes: float64(2), object(8)
memory usage: 54.8+ KB


In [22]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   branch          300 non-null    object 
 1   city            300 non-null    object 
 2   customer_type   300 non-null    object 
 3   gender          300 non-null    object 
 4   product_line    300 non-null    object 
 5   payment_method  300 non-null    object 
 6   rating          300 non-null    float64
 7   time_of_day     300 non-null    object 
 8   day_name        300 non-null    object 
dtypes: float64(1), object(8)
memory usage: 21.2+ KB


In [24]:
target = train.pop("total")

In [25]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)


In [26]:
from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(
    train, target, test_size=0.2, random_state=0
)

In [35]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import root_mean_squared_error

In [ ]:
# 374016.3661648523
lr = LinearRegression()
lr.fit(X_tr,y_tr)
pred = lr.predict(X_val)
print(root_mean_squared_error(y_val, pred))

374016.3661648523


In [ ]:
# 589083.1006353247
dt = DecisionTreeRegressor()
dt.fit(X_tr,y_tr)
pred = dt.predict(X_val)
print(root_mean_squared_error(y_val, pred))

589083.1006353247


In [41]:
# 384295.5066394778
rt = RandomForestRegressor()
rt.fit(X_tr,y_tr)
pred = rt.predict(X_val)
print(root_mean_squared_error(y_val, pred))

386109.8589783684


In [42]:
pred = lr.predict(test)
result = pd.DataFrame({
    'pred':pred
})

result

,pred
0,485484.574028
1,474683.416538
2,486669.157453
3,426407.822229
4,599942.904214
...,...
295,501132.588740
296,554273.922120
297,540502.754963
298,550704.882161


#### 성능 개선


In [43]:
import pandas as pd
train = pd.read_csv('./ch7_data/mart_train.csv')
test = pd.read_csv('./ch7_data/mart_test.csv')

In [44]:
target = train.pop('total')

In [49]:
from sklearn.preprocessing import LabelEncoder

cols = train.select_dtypes(include='O').columns

for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

In [50]:
from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(
    train, target, test_size=0.2, random_state=0
)

In [51]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import root_mean_squared_error

In [ ]:
# 374016.3661648523
# 371070.1344734549
lr = LinearRegression()
lr.fit(X_tr,y_tr)
pred = lr.predict(X_val)
print(root_mean_squared_error(y_val, pred))

371070.1344734549


In [ ]:
# 589083.1006353247
# 576797.5384214957
dt = DecisionTreeRegressor()
dt.fit(X_tr,y_tr)
pred = dt.predict(X_val)
print(root_mean_squared_error(y_val, pred))

576797.5384214957


In [55]:
# 384295.5066394778
# 396410.10123629577
rt = RandomForestRegressor()
rt.fit(X_tr,y_tr)
pred = rt.predict(X_val)
print(root_mean_squared_error(y_val, pred))

399474.0486467783


In [56]:
pred = lr.predict(test)
result = pd.DataFrame({
    'pred':pred
})

result

,pred
0,511234.131202
1,430822.632012
2,490155.030454
3,418695.675057
4,563949.989762
...,...
295,498973.208100
296,512598.176692
297,499254.959659
298,524293.058966


## 3 유형


### 문제
조개 정보를 나타낸 데이터다. 주어진 데이터에서 300개 중 앞에서부터 210개는 train, 90개는 test 데이터로 나눈다.<br>
다중 로지스틱 회귀를 써서 gender(성별)을 예측하되, 패널티는부과하지 않음.

In [57]:
import pandas as pd
data = pd.read_csv('./ch7_data/clam.csv')

In [58]:
data

,age,length,diameter,height,weight,gender
0,6,0.474627,0.211352,0.178189,78.971766,1
1,1,0.465847,0.339388,0.170522,98.781960,1
2,4,0.122807,0.238691,0.106924,88.792625,0
3,4,0.204579,0.360543,0.034261,1.028847,0
4,8,0.243458,0.358037,0.128080,6.503367,0
...,...,...,...,...,...,...
295,3,0.559766,0.390519,0.079062,57.877344,0
296,8,0.371229,0.355306,0.040285,31.909257,1
297,2,0.595571,0.104829,0.132500,4.924550,1
298,7,0.291551,0.289444,0.073238,51.474851,0


In [60]:
train = data.iloc[:210]
test = data.iloc[210:]

#### 1번 문제
- weight를 독립 변수, gender를 종속변수로 사용해 로지스틱 회귀 모델을 만들고,<br>
weight 변수가 무게가 한 단위 증가할 때 수컷일 오즈비 값은(반올림하여 넷째 자리까지 계산)


In [66]:
import numpy as np
from statsmodels.formula.api import logit
model = logit('gender ~ weight', train).fit()

odds_ratio = np.exp(model.params['weight'])
print(round(odds_ratio,4))

Optimization terminated successfully.
         Current function value: 0.690045
         Iterations 4
1.0047


#### 2번 문제
- gender를 종속변수로 하고 나머지 변수들(age,length,diameter,height,weight)을 독립변수로 사용하는<br>
로지스틱 회귀 모델을 적합시킨 후 잔차 이탈도를 계산하시오.

In [70]:
import numpy as np
from statsmodels.formula.api import logit
formula = 'gender ~ age+length+diameter+height+weight'
model =logit(formula, train).fit()

print(round(-2*model.llf,2))

Optimization terminated successfully.
         Current function value: 0.683173
         Iterations 4
286.93


#### 3번 문제
- 독립변수 weight만 사용해 학습한 모델에서 test 데이터의 gender를 예측하고, Error Rate를 구하시오.

In [72]:
from sklearn.metrics import accuracy_score

target = test.pop('gender')

model = logit('gender ~ weight', train).fit()

Optimization terminated successfully.
         Current function value: 0.690045
         Iterations 4


In [74]:
pred = model.predict(test)
pred = (pred>0.5).astype(int)

In [76]:
accuracy = accuracy_score(target,pred)
error_rate = 1-accuracy

print(round(error_rate,3))

0.478


### 문제
- 선형회귀

In [77]:
import pandas as pd 
df = pd.read_csv('./ch7_data/system_cpu.csv')
df

,ERP,Feature1,Feature2,Feature3,CPU
0,30.6,235.1,44.5,44.0,112.3
1,40.3,36.6,46.4,36.1,58.6
2,57.7,52.2,66.5,2.0,55.3
3,128.3,196.2,59.8,57.4,103.2
4,80.3,75.2,59.6,58.2,104.1
...,...,...,...,...,...
110,207.9,178.6,32.4,118.7,140.5
111,150.7,215.0,8.3,80.3,59.2
112,186.3,68.0,-7.2,92.9,82.0
113,141.0,73.0,158.5,82.4,145.0


#### 1번 문제
- ERP와 가장 상관 관계가 높은 값을 구하시오.<br>
(반올림하여 소수 셋째 자리까지 계산)

In [89]:
df_corr = df.corr()['ERP'].abs()
print(round(df_corr[df_corr[1:].idxmax()],3))


0.882


#### 2번 문제
- CPU 컬럼이 100미만인 것만 찾아 ERP를 종속변수, 나머지 변수들을 독립변수로 설정해 선형회귀모델을 만들고 <br>
적합한 결정 계수를 구하시오.(반올림하여 소수 셋째자리까지 계산)


In [ ]:
cond = df['CPU']<100
df = df[cond]

In [92]:
df.columns

Index(['ERP', 'Feature1', 'Feature2', 'Feature3', 'CPU'], dtype='object')

In [100]:
from statsmodels.formula.api import ols
formula = 'ERP ~ Feature1 + Feature2 + Feature3+CPU'
model = ols(formula, df).fit()
print(round(model.rsquared,3))

0.755


#### 3번 문제
- 문제 2-2에서 만든 모델에서 독립변수 중 p-value가 가장 높은 값을 구하시오<br>
(반올림하여 소수 셋째자리까지 계산)


In [105]:
model.summary()

#0.684

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ERP   R-squared:                       0.755
Model:                            OLS   Adj. R-squared:                  0.736
Method:                 Least Squares   F-statistic:                     39.30
Date:                Mon, 24 Nov 2025   Prob (F-statistic):           5.36e-15
Time:                        22:32:13   Log-Likelihood:                -260.40
No. Observations:                  56   AIC:                             530.8
Df Residuals:                      51   BIC:                             540.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     51.4133     19.112      2.690      0.010      13.045      89.782
Feature1      -0.0242      0.059     -0.409      0.684      -0.143       0.094
Feature2       0.0602      0.106      0.569      0.572      -0.152       0.273
Feature3       1.4126      0.113     12.458      0.000       1.185       1.640
CPU           -0.4651      0.234     -1.985      0.053      -0.936       0.005
==============================================================================
Omnibus:                        3.758   Durbin-Watson:                   1.762
Prob(Omnibus):                  0.153   Jarque-Bera (JB):                2.757
Skew:                           0.436   Prob(JB):                        0.252
Kurtosis:                       3.648   Cond. No.                         780.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# 제 8회 기출 문제


## 1유형

### 문제
1) 대륙별 맥주 소비량('beer_serving')의 평균을 계산하고, 평균이 가장 큰 대륙을 찾으세요.
2) 1번에서 찾은 대륙에서 맥주 소비량이 5번째로 많은 국가('country')의 맥주 소비량을 구하시오.


In [111]:
import pandas as pd
df = pd.read_csv('./ch8_data/drinks.csv')
df

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,Asia
1,Albania,89,132,54,4.9,Europe
2,Algeria,25,0,14,0.7,Africa
3,Andorra,245,138,312,12.4,Europe
4,Angola,217,57,45,5.9,Africa
...,...,...,...,...,...,...
188,Venezuela,333,100,3,7.7,South America
189,Vietnam,111,2,1,2.0,Asia
190,Yemen,6,0,0,0.1,Asia
191,Zambia,32,19,4,2.5,Africa


In [140]:
test = df.groupby('continent')['beer_servings'].mean()
test

continent
Europe    106.160622
Name: beer_servings, dtype: float64

In [146]:
top_continent = test.idxmax()
cond = df['continent']==top_continent
df = df[cond]

df = df.sort_values('beer_servings',ascending=False)
df.iloc[4,1]

np.int64(343)

### 문제
1) 관광객비율이 두 번째로 높은 국가의 '사업' 방문객 수를 a라고 정의
2) '관광'이 두번 째로 높은 국가의 '공무' 방문객 수를 b라고 정의
3) a+b를 구하시오.

In [7]:
import pandas as pd
df = pd.read_csv("./ch8_data/tourist.csv")
df

,나라,관광,공무,사업,기타
0,국가1,1184,270,380,55
1,국가2,1059,184,267,86
2,국가3,1129,168,261,50
3,국가4,692,106,214,125
4,국가5,1335,296,296,84
...,...,...,...,...,...
95,국가96,898,119,244,132
96,국가97,1111,195,234,118
97,국가98,1065,172,362,72
98,국가99,1408,254,288,149


In [24]:
# df['방문객 합계'] = ㅇ
df['방문객 합계'] = df['관광'] + df['공무'] + df['사업'] + df['기타']

In [25]:
df['관광객 비율'] = df['관광']/df['방문객 합계']

In [36]:
a = df.sort_values('관광객 비율',ascending=False).iloc[1]['사업']
b = df.sort_values('관광',ascending=False).iloc[1]['공무']
print(a+b)

441


### 문제
1. 주어진 데이터에서 'co','nmhc' 컬럼을 각각 Min-Max 스케일링하시오.
2. 스케일링 된 'co','nmhc' 컬럼의 표준편차를 각각 구하시오.
3. 'co' 컬럼의 표준편차에서 'nmhc' 컬럼의 표준편차를 뺀 값을 소수점 3자리로 반올해서 구하시오.

In [37]:
import pandas as pd
df = pd.read_csv("./ch8_data/chem.csv")
df

,sample,co,nmhc,etc
0,샘플1,79,54,31
1,샘플2,84,57,58
2,샘플3,109,74,113
3,샘플4,15,77,21
4,샘플5,65,77,115
...,...,...,...,...
95,샘플96,56,58,86
96,샘플97,33,59,111
97,샘플98,55,79,105
98,샘플99,69,51,59


In [40]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df['co_scaled'] = scaler.fit_transform(df[['co']])
df['nmhc_scaled'] = scaler.fit_transform(df[['nmhc']])

In [41]:
co_std = df['co_scaled'].std()
nmhc_std = df['nmhc_scaled'].std()

print(co_std,nmhc_std)

std_diff = round(co_std-nmhc_std, 3)
print(std_diff)

0.2856516497116944 0.3030617020578397
-0.017


## 2유형


### 문제
- 통신사에서 고객에게 청구될 총 금액을 예측하시오.
- TotalCharges(총 청구액)
- 평가지표 : MAE

In [65]:
import pandas as pd
train = pd.read_csv("./ch8_data/churn_train.csv")
test = pd.read_csv("./ch8_data/churn_test.csv")

train.shape, test.shape


((4116, 19), (1764, 18))

In [66]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4116 entries, 0 to 4115
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        4116 non-null   object 
 1   gender            4116 non-null   object 
 2   SeniorCitizen     4116 non-null   int64  
 3   Partner           4116 non-null   object 
 4   Dependents        4116 non-null   object 
 5   tenure            4116 non-null   int64  
 6   PhoneService      4116 non-null   object 
 7   MultipleLines     4116 non-null   object 
 8   InternetService   4116 non-null   object 
 9   OnlineSecurity    4116 non-null   object 
 10  OnlineBackup      4116 non-null   object 
 11  DeviceProtection  4116 non-null   object 
 12  TechSupport       4116 non-null   object 
 13  StreamingTV       4116 non-null   object 
 14  StreamingMovies   4116 non-null   object 
 15  Contract          4116 non-null   object 
 16  PaperlessBilling  4116 non-null   object 


In [67]:
# train = pd.get_dummies(train)
# test = pd.get_dummies(test)

In [68]:
train = train.drop('customerID',axis=1)
test = test.drop('customerID',axis=1)
target = train.pop('TotalCharges')

In [69]:
from sklearn.preprocessing import LabelEncoder
cols = train.select_dtypes(include='O').columns
cols
for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

In [70]:
from sklearn.model_selection import train_test_split
X_tr, X_val,y_tr,y_val = train_test_split(
    train,target, test_size = 0.2, random_state =0
)

In [71]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error


In [ ]:
# 914.6588823815637
# 913.0632659267819
lr = LinearRegression()
lr.fit(X_tr,y_tr)
pred = lr.predict(X_val)
print(mean_absolute_error(y_val, pred))

913.0632659267819


In [ ]:
# 1145.2594781553398
# 1244.0059041262136

dt = DecisionTreeRegressor()
dt.fit(X_tr,y_tr)
pred = dt.predict(X_val)
print(mean_absolute_error(y_val, pred))

1244.0059041262136


In [ ]:
# 939.4427709951456
# 943.7515926048891
rf = RandomForestRegressor()
rf.fit(X_tr,y_tr)
pred = rf.predict(X_val)
print(mean_absolute_error(y_val, pred))

943.7515926048891


## 3 유형


1) 주어진 데이터에서 로지스틱 회귀 분석을 수행해 유의확률(p-value)이 0.05 이상인 유의하지 않은 독립변수의 개수를 구하시오.
2) 유의확률(p-value)이 0.05 미만인 유의한 변수만을 사용해 다시 분석을 수행.
이 회귀식의 유의한 회귀계수의 합계를 구하시오.(상수항 포함)
3) 1-2에서 수행한 로지스틱 회귀식에서 'DataUsage' 변수가 5만큼 증가할 때 오즈비를 구하시오.

In [76]:
import pandas as pd
df = pd.read_csv('./ch8_data/churn.csv')
df

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0,51,1,0,1.0,0,303.7,83,72.5,9.3,3.8
1,1,61,1,0,-0.2,2,194.8,89,66.3,10.9,13.3
2,0,84,1,1,-1.3,2,157.6,109,35.7,10.0,12.5
3,0,65,1,0,1.0,1,223.1,129,64.2,9.1,5.1
4,0,154,1,0,1.0,0,137.0,115,40.0,7.7,4.6
...,...,...,...,...,...,...,...,...,...,...,...
995,0,32,1,0,0.6,1,241.2,94,64.1,8.5,13.4
996,0,24,0,0,-1.1,1,155.2,97,48.5,9.8,8.3
997,0,67,1,0,1.9,3,184.6,69,45.8,7.3,8.5
998,0,72,1,0,1.0,0,153.2,65,34.8,9.6,14.5


In [93]:
from statsmodels.formula.api import logit
formula = "Churn ~ AccountWeeks+ContractRenewal+DataPlan+DataUsage+CustServCalls+DayMins+DayCalls+MonthlyCharge+OverageFee+RoamMins"

model = logit(formula, df).fit()

num_list = model.pvalues<0.05
num_list.sum()
print(num_list.sum())

Optimization terminated successfully.
         Current function value: 0.393603
         Iterations 6
3


In [98]:
formula = "Churn ~ DataUsage + DayMins"
model2 = logit(formula,df).fit()
model2.summary()
round(sum(model2.params),3)

Optimization terminated successfully.
         Current function value: 0.397599
         Iterations 6


-1.213

In [99]:
import numpy as np
coef = model.params['DataUsage']
round(np.exp(coef*5),3)

np.float64(0.428)

1) 주어진 데이터를 이용해 종속변수(PIQ)와 독립변수(Brain,Height,Weight)로 다중 선형 회귀 분석을 수행하시오. 이때 유의확률이 가장 작은 변수의 회귀 계수 값을 구하시오.
2) 문제 2-1에 적합한 모델의 결정 계수 값을 구하시오.
3) 뇌 크기가 90, 키가 70, 몸무게가 150일 때 PIQ를 예측하시오.

In [2]:
import pandas as pd
df = pd.read_csv('./ch8_data/piq.csv')

df.head()

,PIQ,Brain,Height,Weight
0,132,85.78,62.5,127
1,96,86.54,68.0,135
2,84,90.49,66.3,134
3,134,79.06,62.0,122
4,86,88.91,70.0,180


In [3]:
from statsmodels.formula.api import ols
model= ols('PIQ ~ Brain+Height+Weight', df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    PIQ   R-squared:                       0.370
Model:                            OLS   Adj. R-squared:                  0.329
Method:                 Least Squares   F-statistic:                     9.024
Date:                Fri, 28 Nov 2025   Prob (F-statistic):           8.25e-05
Time:                        15:54:20   Log-Likelihood:                -217.27
No. Observations:                  50   AIC:                             442.5
Df Residuals:                      46   BIC:                             450.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     74.5114     50.564      1.474      0.1

In [4]:
model.params

print(2.343130)

2.34313


In [5]:
# 2번

print(round(model.rsquared,2))

0.37


In [6]:
# 뇌 크기가 90, 키가 70, 몸무게가 150일 때 PIQ를 예측하시오.
new_data = pd.DataFrame({
    'Brain':[90],
    "Height":[70],
    "Weight":[150]
})

new_data

,Brain,Height,Weight
0,90,70,150


In [7]:
pred = model.predict(new_data)
print(pred)
print(round(pred))

0    106.38302
dtype: float64
0    106.0
dtype: float64


# 제 9회 기출문제


## 1유형


# 문제
1) 총대출액은 신용대출과 담보대출의 합이다.
2) 각 지역코드 내에서, 성별(1과 2) 별로 '총대출액'의 합계를 계산하시오.
3) 성별 간 총대출액의 차이가 가장 큰 지역코드를 구하시오.


In [8]:
import pandas as pd
df = pd.read_csv('./ch9_data/loan.csv')
df

,지역코드,성별,신용대출,담보대출
0,4100000000,1,688608,5706058
1,4100000000,2,611035,2260521
2,4100000001,1,125392,5946493
3,4100000001,2,239679,4711962
4,4100000002,1,623758,5411745
...,...,...,...,...
4995,4100000511,1,104176,8757588
4996,4100000157,2,321815,6782991
4997,4100000309,2,519114,6318150
4998,4100000341,1,106840,4103058


In [9]:
df['총대출액'] = df['신용대출'] + df['담보대출']

In [10]:
df

,지역코드,성별,신용대출,담보대출,총대출액
0,4100000000,1,688608,5706058,6394666
1,4100000000,2,611035,2260521,2871556
2,4100000001,1,125392,5946493,6071885
3,4100000001,2,239679,4711962,4951641
4,4100000002,1,623758,5411745,6035503
...,...,...,...,...,...
4995,4100000511,1,104176,8757588,8861764
4996,4100000157,2,321815,6782991,7104806
4997,4100000309,2,519114,6318150,6837264
4998,4100000341,1,106840,4103058,4209898


In [18]:
grouped = df.groupby(['지역코드','성별'])['총대출액'].sum().unstack()

In [19]:
grouped['차이'] = abs(grouped[1]-grouped[2])

In [21]:
result = grouped['차이'].idxmax()
result

np.int64(4100000278)

# 문제
1) 검거율은 검거건수 / 발생건수
2) 각 연도에서 가장 높은 검거율을 기록한 범죄유형
3) 해당 범죄유형의 검거건수를 구한 후, 이 값들을 모두 합산하시오.


In [27]:
import pandas as pd
df = pd.read_csv('./ch9_data/crime.csv')
df

,연도,구분,강력범죄,절도범죄,폭력범죄,지능범죄,풍속범죄,특별경제범죄,마약범죄,보건범죄,환경범죄,교통범죄,노동범죄,선거범죄,병역범죄,기타범죄
0,2014,발생건수,800,900,750,650,700,1034,600,850,800,900,750,800,700,1000
1,2014,검거건수,500,600,400,450,400,961,350,500,450,550,400,450,400,600
2,2015,발생건수,900,768,800,750,700,1000,850,800,900,950,800,850,750,1000
3,2015,검거건수,600,713,500,550,400,600,500,450,550,600,450,500,400,600
4,2016,발생건수,850,800,880,750,700,950,900,800,850,900,750,800,700,950
5,2016,검거건수,500,600,812,550,400,600,500,450,500,550,400,450,400,600
6,2017,발생건수,1000,900,850,800,750,950,1400,900,1000,950,800,850,750,900
7,2017,검거건수,600,500,500,550,400,600,1300,500,550,600,450,500,400,500
8,2018,발생건수,1450,900,850,800,750,950,900,800,850,900,750,800,700,950
9,2018,검거건수,1350,600,500,550,400,600,500,450,500,550,400,450,400,600


In [28]:
cond1 = df['구분'] == '발생건수'
cond2 = df['구분'] == '검거건수'

df1 = df[cond1].iloc[:,2:]
df2 = df[cond2].iloc[:,2:]

In [32]:
df1 = df1.reset_index(drop=True)
df2 = df2.reset_index(drop=True)

In [33]:
df3 = df2/df1

In [34]:
df3

,강력범죄,절도범죄,폭력범죄,지능범죄,풍속범죄,특별경제범죄,마약범죄,보건범죄,환경범죄,교통범죄,노동범죄,선거범죄,병역범죄,기타범죄
0,0.625000,0.666667,0.533333,0.692308,0.571429,0.929400,0.583333,0.588235,0.562500,0.611111,0.533333,0.562500,0.571429,0.600000
1,0.666667,0.928385,0.625000,0.733333,0.571429,0.600000,0.588235,0.562500,0.611111,0.631579,0.562500,0.588235,0.533333,0.600000
2,0.588235,0.750000,0.922727,0.733333,0.571429,0.631579,0.555556,0.562500,0.588235,0.611111,0.533333,0.562500,0.571429,0.631579
3,0.600000,0.555556,0.588235,0.687500,0.533333,0.631579,0.928571,0.555556,0.550000,0.631579,0.562500,0.588235,0.533333,0.555556
4,0.931034,0.666667,0.588235,0.687500,0.533333,0.631579,0.555556,0.562500,0.588235,0.611111,0.533333,0.562500,0.571429,0.631579
5,0.600000,0.555556,0.588235,0.687500,0.533333,0.631579,0.555556,0.928571,0.550000,0.631579,0.562500,0.588235,0.533333,0.555556
6,0.600000,0.555556,0.588235,0.908667,0.533333,0.631579,0.555556,0.562500,0.588235,0.611111,0.533333,0.562500,0.571429,0.631579


In [35]:
listbox = df3.idxmax(axis=1)

In [36]:
listbox

0    특별경제범죄
1      절도범죄
2      폭력범죄
3      마약범죄
4      강력범죄
5      보건범죄
6      지능범죄
dtype: object

In [37]:
result = 0
for index, item in enumerate(listbox):
    result = result + df2.loc[index,item]
    print(result)

961
1674
2486
3786
5136
6436
7799


In [46]:
# 풀이 2

df_long = pd.melt(df,id_vars=['연도','구분'],var_name='범죄유형',value_name='건수')
df_pivot = df_long.pivot_table(index=['연도','범죄유형'],columns='구분',values='건수').reset_index()
df_pivot['검거율'] = df_pivot['검거건수'] / df_pivot['발생건수']

listbox = df_pivot.groupby('연도')['검거율'].idxmax()
max_crime = df_pivot.loc[listbox]

result = max_crime['검거건수'].sum()
result

np.float64(7799.0)

# 문제

1) 만족도의 결측치는 만족도의 평균값으로 대체
2) 근속 연수의 결측치는 같은 부서 내에서 동일한 성과등급을 가진 직원들의 평균 근속 연수로 대체
    - 이때 평균 근속연수는 소수점 이하를 절사하여 정수로 변환하여 사용
3) 각 직원의 '연봉/근속연수' 값을 계산하여, 그 값이 세번째로 큰 직원의 근속연수를 구하시오.
4) 각 직원의 '연봉/만족도'값을 계산하여, 그 값이 두번째로 큰 직원의 교육 참가 횟수를 구하시오.
5) (3)과 (4)의 값을 더한 결과를 구하시오.

In [48]:
import pandas as pd
df = pd.read_csv("./ch9_data/hr.csv")
df

,사원번호,부서,성과등급,연봉,근속연수,교육참가횟수,만족도
0,E0001,Marketing,B,73200000,14.0,5,6.0
1,E0002,IR,A,95100000,3.0,4,9.0
2,E0003,Manager,B,44700000,5.0,2,8.0
3,E0004,IR,B,122100000,16.0,3,8.0
4,E0005,IR,C,66000000,NaN,4,8.0
...,...,...,...,...,...,...,...
995,E0996,Finance,B,90400000,15.0,3,2.0
996,E0997,IR,C,147600000,12.0,5,5.0
997,E0998,Sales,C,95600000,4.0,4,9.0
998,E0999,Sales,C,94400000,8.0,6,1.0


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   사원번호    1000 non-null   object 
 1   부서      1000 non-null   object 
 2   성과등급    1000 non-null   object 
 3   연봉      1000 non-null   int64  
 4   근속연수    952 non-null    float64
 5   교육참가횟수  1000 non-null   int64  
 6   만족도     880 non-null    float64
dtypes: float64(2), int64(2), object(3)
memory usage: 54.8+ KB


In [51]:
# 1) 만족도의 결측치는 만족도의 평균값으로 대체

fill_value = df['만족도'].mean()
df['만족도'].fillna(fill_value)

0      6.0
1      9.0
2      8.0
3      8.0
4      8.0
      ... 
995    2.0
996    5.0
997    9.0
998    1.0
999    5.0
Name: 만족도, Length: 1000, dtype: float64

In [54]:
fill_target = df[df['근속연수'].isnull()]

In [64]:
test = df.groupby(['부서','성과등급'])['근속연수'].transform("mean")

test = test.astype(int)

In [65]:
df['근속연수'] = df['근속연수'].fillna(test)

In [70]:
# 3) 각 직원의 '연봉/근속연수' 값을 계산하여, 그 값이 세번째로 큰 직원의 근속연수를 구하시오.

df['연봉_근속연수'] = df['연봉']/df['근속연수']
df_year = df.nlargest(3,'연봉_근속연수')
A = df_year.iloc[-1]['근속연수']

# 4) 각 직원의 '연봉/만족도'값을 계산하여, 그 값이 두번째로 큰 직원의 교육 참가 횟수를 구하시오.
df['연봉_만족도'] = df['연봉']/df['만족도']
df_like = df.nlargest(2,'연봉_만족도')
B = df_like.iloc[-1]['교육참가횟수']
# 5) (3)과 (4)의 값을 더한 결과를 구하시오.

print(A+B)

7.0


# 2유형


## 문제
- 농약검출여무(0: 미검출 , 1: 검출, 2 : 재검사 필요)
- macro f1 score


In [71]:
import pandas as pd
train = pd.read_csv('./ch9_data/farm_train.csv')
test = pd.read_csv('./ch9_data/farm_test.csv')

In [72]:
train.shape, test.shape

((4000, 9), (1000, 8))

In [73]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   농업면적    4000 non-null   float64
 1   연도      4000 non-null   int64  
 2   지역      4000 non-null   object 
 3   비료사용량   4000 non-null   float64
 4   비료잔여량   4000 non-null   float64
 5   작물종류    4000 non-null   object 
 6   토양유형    4000 non-null   object 
 7   농약검출여부  4000 non-null   int64  
 8   등급      4000 non-null   object 
dtypes: float64(3), int64(2), object(4)
memory usage: 281.4+ KB


In [74]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   농업면적    1000 non-null   float64
 1   연도      1000 non-null   int64  
 2   지역      1000 non-null   object 
 3   비료사용량   1000 non-null   float64
 4   비료잔여량   1000 non-null   float64
 5   작물종류    1000 non-null   object 
 6   토양유형    1000 non-null   object 
 7   등급      1000 non-null   object 
dtypes: float64(3), int64(1), object(4)
memory usage: 62.6+ KB


In [75]:
train['농약검출여부'].value_counts()

농약검출여부
2    1989
0    1758
1     253
Name: count, dtype: int64

In [77]:
target = train.pop('농약검출여부')

In [79]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [81]:
from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(
    train,target, test_size = 0.2, random_state = 0
)



In [91]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import f1_score

In [97]:
dt = DecisionTreeClassifier(random_state=0)
model = dt.fit(X_tr,y_tr)
pred =dt.predict(X_val)

print(f1_score(y_val,pred, average='macro'))

0.8917625730999603


In [98]:
rf = RandomForestClassifier(random_state=0)
model = rf.fit(X_tr,y_tr)
pred =rf.predict(X_val)
print(f1_score(y_val,pred,average='macro'))

0.8462889767237595


In [101]:
pred = rf.predict(test)
submit = pd.DataFrame({
    "pred":pred
})

# 3유형

## 문제
- 주어진 데이터 id를 기준으로 다음과 같이 데이터를 분할하여 소문제 해결
- train 데이터 1 <= id <= 140
- test 데이터 : id > 140


In [105]:
import pandas as pd
df = pd.read_csv('./ch9_data/design.csv')
df

,id,design,c1,c2,c3,c4,c5
0,1,58.962471,0.374540,0.642032,0.103124,0.168935,0.707239
1,2,42.006534,0.950714,0.084140,0.902553,0.278590,0.152539
2,3,55.831980,0.731994,0.161629,0.505252,0.177010,0.576288
3,4,64.458592,0.598658,0.898554,0.826457,0.088703,0.606715
4,5,61.342792,0.156019,0.606429,0.320050,0.120636,0.424131
...,...,...,...,...,...,...,...
195,196,49.976045,0.349210,0.930757,0.473962,0.872124,0.091582
196,197,61.148418,0.725956,0.858413,0.667558,0.932118,0.917314
197,198,38.647434,0.897110,0.428994,0.172320,0.565133,0.136819
198,199,49.458373,0.887086,0.750871,0.192289,0.696651,0.950237


In [107]:
train = df.iloc[:140]
test = df.iloc[140:]

1. train 데이터에서 c1,c2,c3,c4,c5를 독립변수로, design을 종속변수로 하는 다중 회귀 모형을 적합한 후, 절편을 제외한 독립변수중 P-value가 0.05 보다 작은 변수 개수
2. 1-1에서 적합한 회귀모형의 결과를 바탕으로 p-value가 0.05 이하인 유의한 독립변수만을 선택하여 새로운 다중 회귀모형을 적합한 후
    <br>train 데이터에서 design의 예측값을 산출하고 예측값과 실제 design 값 사이의 피어슨 상관계수를 반올림하여 소수 셋째자리까지 계산
3. 1-2 에서 적합한 회귀 모형을 이용하여 test 데이터에서 design의 예측값을 산출한 후, 예측값과 실제 design 값 사이의 rmse를 반올림하여 계산


In [116]:
from statsmodels.formula.api import ols

formula = 'design ~ c1+c2+c3+c4+c5'
model = ols(formula,train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 design   R-squared:                       0.266
Model:                            OLS   Adj. R-squared:                  0.238
Method:                 Least Squares   F-statistic:                     9.697
Date:                Fri, 28 Nov 2025   Prob (F-statistic):           6.37e-08
Time:                        17:14:25   Log-Likelihood:                -468.72
No. Observations:                 140   AIC:                             949.4
Df Residuals:                     134   BIC:                             967.1
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     53.0220      2.294     23.112      0.000      48.485      57.559
c1            -4.9403      2.008     -2.460      0.015      -8.912      -0.969
c2            11.3795      2.045      5.564      0.000       7.335      15.424
c3             2.6960      1.920      1.404      0.163      -1.101       6.493
c4             5.7978      2.176      2.664      0.009       1.494      10.102
c5            -1.5018      2.067     -0.726      0.469      -5.590       2.587
==============================================================================
Omnibus:                        3.969   Durbin-Watson:                   2.052
Prob(Omnibus):                  0.137   Jarque-Bera (JB):                3.019
Skew:                           0.227   Prob(JB):                        0.221
Kurtosis:                       2.441   Cond. No.                         7.68
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
print(model.pvalues[1:]<0.05)

# 3개

c1     True
c2     True
c3    False
c4     True
c5    False
dtype: bool


In [119]:
formula = 'design ~ c1+c2+c4'

model = ols(formula, train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 design   R-squared:                       0.251
Model:                            OLS   Adj. R-squared:                  0.234
Method:                 Least Squares   F-statistic:                     15.17
Date:                Fri, 28 Nov 2025   Prob (F-statistic):           1.43e-08
Time:                        17:16:03   Log-Likelihood:                -470.14
No. Observations:                 140   AIC:                             948.3
Df Residuals:                     136   BIC:                             960.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     53.6637      1.808     29.686      0.000      50.089      57.239
c1            -4.8767      2.010     -2.426      0.017      -8.852      -0.901
c2            11.1863      2.039      5.487      0.000       7.154      15.218
c4             6.2272      2.159      2.885      0.005       1.958      10.496
==============================================================================
Omnibus:                        3.056   Durbin-Watson:                   1.972
Prob(Omnibus):                  0.217   Jarque-Bera (JB):                2.947
Skew:                           0.296   Prob(JB):                        0.229
Kurtosis:                       2.606   Cond. No.                         5.96
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [120]:
train['pred_design'] = model.predict(train)

/var/folders/6c/5zw1q8hx6r51frqls4xy7vdc0000gn/T/ipykernel_8867/3786641288.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['pred_design'] = model.predict(train)


In [122]:
result = train['design'].corr(train['pred_design'])
print(round(result,3))

0.501


In [123]:
# 1-2 에서 적합한 회귀 모형을 이용하여 test 데이터에서 design의 예측값을 산출한 후, 예측값과 실제 design 값 사이의 rmse를 반올림하여 계산

test['pred_design'] = model.predict(test)

/var/folders/6c/5zw1q8hx6r51frqls4xy7vdc0000gn/T/ipykernel_8867/1576711762.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred_design'] = model.predict(test)


In [126]:
from sklearn.metrics import root_mean_squared_error

In [127]:
print(root_mean_squared_error(test['design'],test['pred_design']))

8.488336110545157


## 문제



- 종속병수 : Churn(고객 이탈 여부 : 1, 유지 : 0)

- 독립변수 : 
    - MonthlyCharges(월 사용 요금)
    - CustomerTenure(고객 유지 기간)
    - HasPhoneService(전화 서비스 이용 여부)
    - HasTechInsurance(기술 보험 가입 여부)

In [129]:
import pandas as pd
df = pd.read_csv("./ch9_data/retention.csv")
df

,CustomerID,MonthlyCharges,CustomerTenure,HasPhoneService,HasTechInsurance,Churn
0,1,77.450712,47,1,1,1
1,2,67.926035,68,1,1,0
2,3,79.715328,45,0,1,1
3,4,92.845448,2,1,0,1
4,5,66.487699,27,1,1,0
...,...,...,...,...,...,...
75,76,82.328538,29,1,1,0
76,77,71.305706,63,1,0,1
77,78,65.514890,22,1,0,0
78,79,71.376412,26,0,0,1


1. 주어진 데이터를 사용하여 고객 이탈(Churn)을 예측하는 로지스틱 회귀 모혈을 적합한 후, MonthlyCharges 변수의 p-value를 반올림하여 소수 셋째자리까지 구하시오.
2. 적합한 로지스틱 회귀모형을 바탕으로, 전화 서비스를 받지 않은 고객이 이탈할 확률 대비 전화 서비스를 받은 고객이 이탈할 확률에 대한 오즈비를 계산하시오.

In [142]:
from statsmodels.formula.api import logit

formula = 'Churn ~ MonthlyCharges+CustomerTenure+HasPhoneService+HasTechInsurance'

model = logit(formula, df).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.582234
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  Churn   No. Observations:                   80
Model:                          Logit   Df Residuals:                       75
Method:                           MLE   Df Model:                            4
Date:                Fri, 28 Nov 2025   Pseudo R-squ.:                  0.1585
Time:                        17:32:31   Log-Likelihood:                -46.579
converged:                       True   LL-Null:                       -55.352
Covariance Type:            nonrobust   LLR p-value:                  0.001513
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -4.4731      1.437     -3.114      0.002      -7.289      -1.657
MonthlyCharges       0.0503      0.019      2.640      0.008       0.013       0.088
CustomerTenure       0.0428      0.014      3.016      0.003       0.015       0.071
HasPhoneService     -0.3558      0.525     -0.677      0.498      -1.386       0.674
HasTechInsurance    -0.4868      0.518     -0.940      0.347      -1.502       0.529
====================================================================================
"""

In [143]:
print(round(model.pvalues['MonthlyCharges'],3))

0.008


In [144]:
import numpy as np

result = np.exp(model.params['HasPhoneService'])
round(result,3)

np.float64(0.701)

In [146]:
pred_probs = model.predict(df)

In [147]:
sum(pred_probs > 0.3)

65